In [ ]:
import dotenv from "dotenv";
dotenv.config({ path: '.env.development.local' });
// Deno imports
import { Stagehand } from "@browserbasehq/stagehand";
import { z } from 'zod';

const cdpPort = 9222;
let stagehand = null;

In [ ]:
//Initialize Stagehand
(async () => {
  stagehand = new Stagehand({
    verbose: 1, // Verbosity level for logging: 0 = silent, 1 = info, 2 = all
    domSettleTimeoutMs: 30_000, // Timeout for DOM to settle in milliseconds

    // LLM configuration
    modelName: "gpt-4o", // Name of the model to use
    modelClientOptions: {
      apiKey: process.env.OPENAI_API_KEY,
    }, // Configuration options for the model client

    // Browser configuration
    env: "LOCAL", // Environment to run in: LOCAL or BROWSERBASE
    localBrowserLaunchOptions: {
      cdpUrl: `http://localhost:${cdpPort}`,
      viewport: {
        width: 1024,
        height: 768,
      },
    }, // Configuration options for the local browser
  });
  await stagehand.init();

  console.log("Stagehand initialized successfully!");
})();


In [ ]:
let taskInformation = {
    employerType: 'Commercial',
    hasFEIN: 'Yes',
    idType: 'SSN',
    californiaRegistry: 'Yes',
    organizationType: "Limited Liability Company",
    idNumber: "123-45-6789",
    firstName: "John",
    lastName: "Doe",
    dateOfBirth: "1980-01-01",
    dateOfOwnership: "5/15/2020",
    industryActivity: "Other",
    industryDescription: "Software Development",
    organizationLegalName: "Doe Technologies LLC",
    stateOfIncorporation:"TX",
    fileNumber: "987654321",
    FEIN: "12-9443381",
    businessAddress: {
      country: "USA",
      street: "123 Tech Lane" + " " + "Suite 100",
      city: 'San Francisco',
      state: 'CA',
      zipcode: '94114'
    },
    mailingAddress: { // Using Lettuce Address for our customers.
      country: 'USA',
      street: '2261 Market Street STE 22617',
      city: 'San Francisco',
      state: 'CA',
      zipcode: '94114'
    },
    businessPhoneNumber: "555-123-4567",
    businessEmail: "john.doe@example.com"
  }

In [ ]:

//This import would be in index.js
import OpenAI from "openai";

//This would be needed as an argument when imported from utils
const openai = new OpenAI({
  apiKey: process.env.OPENAI_API_KEY,
});

/**
 * Asks ChatGPT which options apply based on the description and question,
 * returning an array of indices for "select all that apply" questions.
 * 
 * @param {string} description - Business description.
 * @param {string} question - The question to evaluate.
 * @param {Array<string>} options - Array of option descriptions.
 * @returns {Promise<Array<number>>} - Array of selected option indices.
 */
export async function multipleOptionGPT(description, question, options) {
  let prompt = `I have the following question and business description. 
  Your task is to determine ALL options that correctly apply based on the description.

  Respond ONLY with the NUMBERS (starting at 0) of ALL correct options, separated by commas, in ascending order, with no spaces or explanations.

  Example valid responses: 0,2 or 1,3,4

  Description: ${description}

  Question: ${question}

  Options:
  `;

  options.forEach((opt, idx) => {
    prompt += `${idx}: ${opt}\n`;
  });

  const completion = await openai.chat.completions.create({
    model: "gpt-4o",
    messages: [
      { role: "system", content: "You are a helpful assistant." },
      { role: "user", content: prompt }
    ]
  });

  const rawAnswer = completion.choices[0].message.content.trim();

  // Parse response into array of indices
  const matches = rawAnswer.match(/\d+/g);
  if (!matches) {
    throw new Error(`Could not parse valid option numbers from GPT response: "${rawAnswer}"`);
  }

  const indices = matches.map(str => {
    const n = parseInt(str, 10);
    if (n < 0 || n >= options.length) {
      throw new Error(`Returned index ${n} out of bounds for options length ${options.length}`);
    }
    return n;
  });

  // Remove duplicates & sort
  const uniqueSorted = Array.from(new Set(indices)).sort((a, b) => a - b);

  return uniqueSorted;
}

In [ ]:

/**
 * Asks ChatGPT which option is correct based on the description and question.
 * @param {string} description - Business description.
 * @param {string} question - The question to evaluate.
 * @param {Array<string>} options - Array of option descriptions.
 * @returns {Promise<number>} - The index of the most correct option.
 */
export async function askGptForOption(description, question, options) {
  let prompt = `I have the following question and business description. 
Your task is to ONLY answer with the NUMBER (starting at 0) of the most correct option based on the description.
Do not explain your answer. Respond ONLY with the number, nothing else.

Description: ${description}

Question: ${question}

Options:
`;

  options.forEach((opt, idx) => {
    prompt += `${idx}: ${opt}\n`;
  });

  const completion = await openai.chat.completions.create({
    model: "gpt-4o",
    messages: [
      { role: "system", content: "You are a helpful assistant." },
      { role: "user", content: prompt }
    ]
  });

  const rawAnswer = completion.choices[0].message.content.trim();

  // Extract integer robustly
  const match = rawAnswer.match(/\d+/);
  if (!match) {
    throw new Error(`Could not parse a valid option number from GPT response: "${rawAnswer}"`);
  }

  const index = parseInt(match[0], 10);
  if (index < 0 || index >= options.length) {
    throw new Error(`Returned index ${index} out of bounds for options length ${options.length}`);
  }

  return index;
}

In [ ]:
async function askYesNoGPT(description, question) {

  const prompt = `You will receive a business description and a Yes/No question that relates to the nature of the business. Answer strictly with:

  "0" if the correct answer is Yes.
  "1" if the correct answer is No.

  And explain the reasoning behind your answer.

  Business Description:
  ${description}

  Question:
  ${question}

  Respond now:`;

  const completion = await openai.chat.completions.create({
    model: "gpt-4o-mini",
    messages: [
      { role: "system", content: "You are a helpful assistant." },
      { role: "user", content: prompt }
    ]
  });

  const rawAnswer = completion.choices[0].message.content.trim();
  console.log(`✅ GPT raw answer: "${rawAnswer}"`);

  if (rawAnswer === "1") return 1;
  if (rawAnswer === "0") return 0;

  // Fallback handling if GPT outputs "Yes"/"No" instead
  const lower = rawAnswer.toLowerCase();
  if (lower.includes("yes")) return 0;
  if (lower.includes("no")) return 1;

  throw new Error(`Invalid GPT response: "${rawAnswer}"`);
}

In [ ]:
(async () => {
    await stagehand.page.goto("https://docs.google.com/forms/d/1mFK8VwdfrO3VLRC8EYqdZ6x5ObVJm9655TmrE-E92kE/edit")
})();

In [ ]:
(async () => {
  try {
      let description = 'Sunshine Spin Laundromat LLC is a Florida-based self-service and drop-off laundry facility providing efficient, high-capacity washers and dryers for individuals and families seeking convenient and reliable laundry solutions. The business offers coin-operated and card payment options, wash-and-fold services, and maintains a clean, air-conditioned environment with free Wi-Fi to enhance customer comfort while they wait. Located in a high-traffic area, Sunshine Spin Laundromat serves local residents, students, and seasonal visitors with extended operating hours to accommodate varied schedules, focusing on delivering quality service and maintaining customer satisfaction.';
      let q = await stagehand.page.extract("The instruction corresponding to the multiple checkboxes available");
      let options = await stagehand.page.observe(`Find all checkboxes for '${q}'`);
      
      console.log(options);
      
      let optionDescriptions = options.map(opt => opt.description);
      let answer = await multipleOptionGPT(description, q, optionDescriptions);

      console.log(answer);
      for (const idx of answer) {
          await stagehand.page.act(options[idx]);
        }
      } catch (error) {
        console.error("❌ Pipeline Error:", error);
  }
})();

In [ ]:
(async () => {
    await stagehand.page.act("Click on the 'Next' or 'Siguiente' button")
})();

In [ ]:
(async () => {
  try {
    let description = 'Sunshine Spin LLC is a Florida-based business where people can get their clothes dry-cleaned.';

    // 🟩 Extract the FIRST Yes/No statement only
    let questionResult = await stagehand.page.extract("Find the first statement corresponding to Yes/No buttons on the page");
    let question = questionResult.extraction.trim();
    console.log("✅ Question extracted:\n", question);

    // 🟩 Observe Yes/No options for that specific question
    let options = await stagehand.page.observe(`Find the Yes/No radio buttons for '${question}'`);
    console.log("✅ Observed options:", options.map(o => o.description));

    // 🟩 Clean the question for GPT clarity
    let cleanedQuestion = question.replace(/\s*-\s*Yes\s*-\s*No\s*$/i, "").trim();

    // 🟩 Use GPT to decide Yes/No, expecting 0 or 1, fallback if necessary
    let gptAnswer = await askYesNoGPT(description, cleanedQuestion);
    console.log(`✅ GPT raw answer: ${gptAnswer}`);

    // 🟩 Act to select the correct radio button using the resolved index
    await stagehand.page.act(options[gptAnswer]);
    console.log(`✅ Selected '${options[gptAnswer].description}' for question: ${cleanedQuestion}`);

  } catch (error) {
    console.error("❌ Pipeline Error:", error);
  }
})();

In [ ]:
(async () => {
    try {
      let description = 'Sunshine Spin Laundromat LLC is a Florida-based self-service and drop-off laundry facility providing efficient, high-capacity washers and dryers for individuals and families seeking convenient and reliable laundry solutions. The business offers coin-operated and card payment options, wash-and-fold services, and maintains a clean, air-conditioned environment with free Wi-Fi to enhance customer comfort while they wait. Located in a high-traffic area, Sunshine Spin Laundromat serves local residents, students, and seasonal visitors with extended operating hours to accommodate varied schedules, focusing on delivering quality service and maintaining customer satisfaction.';
  
      // 🟩 Extract the FIRST Yes/No statement only
      let questionResult = await stagehand.page.extract("Find the second statement corresponding to Yes/No buttons on the page");
      let question = questionResult.extraction.trim();
      console.log("✅ Question extracted:\n", question);
  
      // 🟩 Observe Yes/No options for that specific question
      let options = await stagehand.page.observe(`Find the Yes/No radio buttons for '${question}'`);
      console.log("✅ Observed options:", options.map(o => o.description));
  
      // 🟩 Clean the question for GPT clarity
      let cleanedQuestion = question.replace(/\s*-\s*Yes\s*-\s*No\s*$/i, "").trim();
  
      // 🟩 Use GPT to decide Yes/No, expecting 0 or 1, fallback if necessary
      let gptAnswer = await askYesNoGPT(description, cleanedQuestion);
      console.log(`✅ GPT raw answer: ${gptAnswer}`);
  
      // 🟩 Act to select the correct radio button using the resolved index
      await stagehand.page.act(options[gptAnswer]);
      console.log(`✅ Selected '${options[gptAnswer].description}' for question: ${cleanedQuestion}`);
  
    } catch (error) {
      console.error("❌ Pipeline Error:", error);
    }
  })();

In [ ]:
(async () => {
    await stagehand.page.act("Click on the 'Next' or 'Siguiente' button")
})();

In [ ]:
(async () => {
  try {
    let description = 'Sunshine Spin Laundromat LLC is a Florida-based self-service and drop-off laundry facility providing efficient, high-capacity washers and dryers for individuals and families seeking convenient and reliable laundry solutions. The business offers coin-operated and card payment options, wash-and-fold services, and maintains a clean, air-conditioned environment with free Wi-Fi to enhance customer comfort while they wait. Located in a high-traffic area, Sunshine Spin Laundromat serves local residents, students, and seasonal visitors with extended operating hours to accommodate varied schedules, focusing on delivering quality service and maintaining customer satisfaction.';
    let q = await stagehand.page.extract("The instruction corresponding to the multiple checkboxes available");
    let options = await stagehand.page.observe(`Find all checkboxes for '${q}'`);
    
    let optionDescriptions = options.map(opt => opt.description);
    let answer = await multipleOptionGPT(description, q, optionDescriptions);

    console.log(answer);
    for (const idx of answer) {
        await stagehand.page.act(options[idx]);
      }
  } catch (error) {
    console.error("❌ Pipeline Error:", error);
  }
})();

In [ ]:
(async () => {
    await stagehand.page.act("Click on the 'Next' or 'Siguiente' button")
})();

In [ ]:
//tried using the act method directly, not good

(async () => {
  try {
    let description = 'Sunshine Spin Laundromat LLC is a Florida-based self-service and drop-off laundry facility providing efficient, high-capacity washers and dryers for individuals and families seeking convenient and reliable laundry solutions. The business offers coin-operated and card payment options, wash-and-fold services, and maintains a clean, air-conditioned environment with free Wi-Fi to enhance customer comfort while they wait. Located in a high-traffic area, Sunshine Spin Laundromat serves local residents, students, and seasonal visitors with extended operating hours to accommodate varied schedules, focusing on delivering quality service and maintaining customer satisfaction.';
    let q = await stagehand.page.extract("The instruction corresponding the the multiple checkboxes available");
    let options = await stagehand.page.observe(`Find all checkboxes for '${q}'`);

    let prompt = `I have the following question and business description. 
    Your task is to determine ALL options that correctly apply based on the description.
  
    Click on the options that better fit the description.
  
    Description: ${description}
  
    Question: ${q}
  
    Options:
    `;
  
    options.forEach((opt, idx) => {
      prompt += `${idx}: ${opt}\n`;
    });
    
    // ✅ Pass the ACTUAL `options` directly:
    await stagehand.page.act(prompt);

  } catch (error) {
    console.error("❌ Pipeline Error:", error);
  }
})();
